In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy.polynomial.polynomial as poly

import read_qdp

data4 = pd.read_excel (r'C:\Users\callu\Dropbox\University\Year 3\Block 2b\Project\17D.xls')

ModuleNotFoundError: No module named 'read_qdp'

In [21]:
class TES(object):
    '''
    
    Methods:
    self.plotPV() - plots PV graph
    self.plotIV() - plots IV graph
    self.plotPR() - plots PR graph
    self.plotRegions(raw=1) - set raw to false to plot analysed IV regions
    
    #TODO - Store the temperatures in the object
    '''
    
    def __init__(self, rawIV, params = 0, name='NA'): 
        ''' Creates a TES object from a pandas dataframe. 
        
        Reads the first two coloums of the dataframe and saves 
        them as attributes. Dataframe can consist of more coloums but they are ignored. 
        Params can be a list or tuple of values in this order [R_fb, M_ratio, R_shunt]
        R_stray can be inlcuded as extra paramter. Doing so will prevent the calc_R_stray method being used to 
        generate a value.
        
        Attriutes after initialisation:
        rawI (np.array[floats]), rawV (np.array[floats])
        raw_grad/raw_2grad (list[floats]) - 1st and 2nd differentials of rawIV
        superconducting (bool) - whether the data shows signs of superconductivity
        y_offset (float) 
        I_TES (list[floats]), V_bias (list[floats]) - Calibrated data
        R_fb, M_ratio, R_shunt (floats)- experimental parameters
        gain (float) - calcuated from M_ratio and R_fb
        R_stray (float) - either input or calculated
        P_TES , R_TES (floats) - Power and Resistance values
        rnn (float) 
        
        ohmic_reg, trans_reg, super_reg (ints) the demarcated regions. 
        If not super conducting ohmic_reg will be the entire data range
        '''
        
        # Drop NaNs in spreadsheet to prevent errors later 
        self.rawI = (rawIV.iloc[:,0].dropna()).values #Store as individual np.arrays for more readable code
        self.rawV = (rawIV.iloc[:,1].dropna()).values
        
        # Search for columns with T in the header then refine into BB or bath
        temps = rawIV.filter(regex='T.+', axis = 1)
        T_BB = (temps.filter(regex='BB', axis = 1).dropna()).values
        self.T_BB = [entry[0] for entry in T_BB]
        T_bath = (temps.filter(regex='bath', axis = 1).dropna()).values
        self.T_bath = [entry[0] for entry in T_bath]
        
        self.raw_grad = np.gradient(self.rawV, self.rawI)#Prevents repeated calls to gradient function later
        self.raw_2grad = np.gradient(self.raw_grad, self.rawI)
        self.superconducting = self.calc_superconducting() # This needs to be done first as other inits depend
        
        #iniditalise these values with the methods
        self.name = name
        self.y_offset = 0
        
        # All needed to calculate adjusted I and V etc. If params are included in initialisation, then they are
        # used. Otherwise defualts are used.
        # TODO: raise exception if len is not 3
        if params:
            try:
                self.R_fb = params[0]
                self.M_ratio = params[1]
                self.R_shunt = params[2]
            except IndexError:
                print("Error: Parameters not correct")
                return
        else:
            print("WARNING: Using defualt paramters")
            self.R_fb = 100099.6
            self.M_ratio = 18.9333333333333
            self.R_shunt = 0.00389
        
        self.gain = 1 / (self.R_fb * self.M_ratio)
        
        
    ############################calc_methods############################   
    
    def calc_y_offset_ohmic(self):
        '''
        Input: Threshold percentage (int)
        OutPut: D (float)
        '''
        if self.superconducting:
            self.identify_regions()
            C, D , xvals = self.fit_ohmic(self.rawV, self.rawI)
            self.y_offset = D
            
            self.C = C
            self.D = D
            
            #TODO find out why I did this
            self.rawXvals = xvals
        else:
            C, D = curve_fit(self.f, self.rawV, self.rawI)[0]
            self.y_offset = D
            self.ohmic_reg = [(0, len(self.rawI/2)),(len(self.rawI/2)+1, -1)]
            self.transition_reg = False
            self.super_reg = False
    
    def calc_y_offset(self):
        if self.superconducting:
            self.identify_regions()
            E, F , xvals = self.fit_super(self.rawV, self.rawI)
            self.y_offset = F
            
            self.E = E
            self.F = F
        else:
            C, D = curve_fit(self.f, self.rawV, self.rawI)[0]
            self.y_offset = D
            self.ohmic_reg = [(0, len(self.rawI/2)),(len(self.rawI/2)+1, -1)]
            self.transition_reg = False
            self.super_reg = False
        
    def calc_I_TES(self, tweak=1):
        '''
        Generate I_tes (array) data from tweak(float), y_offset (float), gain (float).
        tweaks not yet used
        '''
        
        self.I_TES = [(entry - self.y_offset) * (10**6) * self.gain for entry in self.rawV]
        
    def calc_V_bias(self, tweak=1):
        '''
        Generates V_bias data
        '''
        self.V_bias = [((self.rawI[i]) * self.R_shunt) - (self.I_TES[i]*(self.R_shunt + self.R_stray)) for i in range(len(self.rawI))]
        
    def calc_P_TES(self):
        self.P_TES = [self.I_TES[index] * self.V_bias[index] for index in range(len(self.I_TES))]
        
    def calc_R_TES(self):
        self.R_TES = [self.V_bias[i] / self.I_TES[i] for i in range(len(self.V_bias))]
        
    def calc_superconducting(self):
        '''
        Test if the gradient deviates from the average by a significant margin. And save result in attribute.
        Returns the boolean value of self.superconducting
        '''
        grad = self.raw_grad
        grad_av = sum(grad)/len(grad)
        if max(grad) > 3*(grad_av) and abs(min(grad)) > 5*(grad_av):
            self.superconducting = True
        else:
            self.superconducting = False
    
        return self.superconducting
    
    def calc_R_stray(self, start = 0 , end = 0.0008, step = 0.0000001):
        '''
        Go through a range of R_strays and find the one which gives the least error when compared to the yaxis. 
        Then set this as self.rstray
        Inputs:
        start, end and step : (floats)
        used in np.arange function that searches for R_stray. So the precision of R_stray value will depend on these.
        threshold: (float)
        The margin around the axis which you want to consider for catching points. 
        '''
        #Initialise lists for storage of results
        resists = []
        scores = []

        for resist in np.arange(start, end, step):
            V_bias = []
            start , stop = self.super_reg[0]
            
            for index in range(start, stop):
                        V_bias.append((self.newI[index] * self.R_shunt) - (self.I_TES[index]*(self.R_shunt + resist)))
            score = 0
            for entry in V_bias:
                    score += abs(entry)**2

            # Scores are errors, make a list so we can find the resistance that has lowest error
            scores.append(score)
            resists.append(resist)
        plt.show()
        plt.plot(resists, scores)
        plt.show()
        self.R_stray = resists[np.argmin(scores)]
    
    ############################utility methods############################   
    
    def f(self, x, A, B):
        ''' 
        Function of straight line for optimisation function to find
        coefficients for.
        '''
        return A*x + B
        
    def identify_regions(self):
        ''' Identifies the different regions of data (ohmic/transition/superconducting) and saves thier indices
        '''
        
        #Check if the data is superconducting, does not proceed if not.
        if not self.superconducting:
            print("Error: TES is not superconducting")
            return 1
        
        ohmic_reg = []
        trans_reg = []
        super_reg = []
        
        index = 0
        ohmic_grad = sum(self.raw_grad[:20])/20
        
        
        #Identify first ohmic region by when the gradient deviates from average of original 
        while True:
            if self.raw_grad[index] < ohmic_grad * 0.95:
                ohmic_reg.append((0, index - 1))
                break

            index += 1

        start = index
        #When the gradient becomes positive once again this indicates superconductivty 
        while True:
            if self.raw_grad[index] > ohmic_grad * 2:
                trans_reg.append((start, index-1))
                break
            index += 1

        start = index
        #When the gradient becomes negative again, second transition region
        while True:
            if self.raw_2grad[index] < min(self.raw_2grad)/10 and self.raw_grad[index] > 0:
                index+=1
                super_reg.append((start,index))
                break
            index += 1

        start = index + 1 
        # No longer trans when the gradient returns to the ohmic
        while True:
            if self.raw_grad[index] > ohmic_grad * 0.99:
                trans_reg.append((start, index))
                break
            index += 1
        
        # The rest of the indices are assumed to be ohmic again
        ohmic_reg.append((index + 1, len(self.raw_grad) - 1))
        
        # Save the reions as attributes in the object
        self.ohmic_reg = ohmic_reg
        self.trans_reg = trans_reg
        self.super_reg = super_reg
        
    def fit_ohmic(self, inputdataX, inputdataY):
        ''' 
        Accepts two lists for X and Y vals and a tuple of indices. These are then used 
        to fit a straight line to the straight regions.
        Returns C, D, xvals. Which are the gradient, intercept and range of the fitted curve.
        '''
    
        r1start, r1end = (0,7)
        r2start, r2end = (-7,-1)
        
        firstx = min(inputdataX)
        lastx = max(inputdataX)
        xvals = np.linspace(firstx, lastx)

        #Slice the data in the frame and store in an array 
        yvals1 = inputdataY[r1start:r1end]
        yvals2 = inputdataY[r2start:r2end]
        lineDataY = np.append(yvals1, yvals2)
        
        xvals1 = inputdataX[r1start:r1end]
        xvals2 = inputdataX[r2start:r2end]
        lineDataX = np.append(xvals1, xvals2)

        #fit the sliced data to a line
        C, D = curve_fit(self.f, lineDataX, lineDataY)[0]

        return C , D , xvals
    
    def fit_super(self, inputdataX, inputdataY):
        ''' 
        Accepts two lists for X and Y vals. These are then used 
        to fit a straight line to the straight superconducting region.
        Returns C, D, xvals. Which are the gradient, intercept and range of the fitted curve.
        '''
    
        super_start, super_end = self.super_reg[0]
        
        firstx = inputdataX[super_start]
        lastx = inputdataX[super_end]
        xvals = np.linspace(firstx, lastx)

        #Slice the data in the frame and store in an array 
        yvals = inputdataY[super_start:super_end]
        xvals = inputdataX[super_start:super_end]
        
        #fit the sliced data to a line
        C, D = curve_fit(self.f, xvals, yvals)[0]

        return C , D , xvals
    
    def power_readout(self):
        p_grad = np.gradient(self.P_TES, self.V_bias)
        constants = []
        for i in range(len(p_grad)):
            if abs(p_grad[i]) < 0.1:
                constants.append(self.P_TES[i])
        if len(constants):
            ans = sum(constants)/len(constants)
        else:
            ans = 0
        return ans
    
    ###############Plotting Methods###################
    def plotIV(self, plot = True):
        '''
        Plots the calibrated IV curve if plot is set to true in the args. Otherwise just calcs R_nn
        '''
        C, D , Axvals = self.fit_ohmic(self.V_bias, self.I_TES)
        self.rnn = 1/C
        if plot:
            fig, ax = plt.subplots()
            plt.plot(Axvals, self.f(Axvals, C , D), color='r', label='Load Line')
            plt.grid()
            plt.scatter(self.V_bias, self.I_TES, label='TES IV')
            ax.set_xlabel("Voltage (mV)")
            ax.set_ylabel("TES Current (mA)")
            plt.title("Calibrated IV Curve")
            ax.legend()
            
    def plotPR(self, name = ''):
        fig, ax = plt.subplots()
        plt.grid()
        plt.plot(self.R_TES, self.P_TES, label='TES PR')#, alpha=0.2)
        plt.title("Power vs Resistance Curve: Index - " + str(name))
        ax.set_xlabel("TES Resistance (ohm)")
        ax.set_ylabel("TES Power (fW)")
        
        if self.calc_superconducting():
            readout = self.power_readout()
            plt.axhline(readout, color='g',label=round(readout,5))
        
        plt.axvline(self.rnn, color='r',label='Rnn: ' + str(round(self.rnn,5)))
        plt.legend()
    
    def plotPV(self):
        fig, ax = plt.subplots()
        plt.grid()
        plt.scatter(self.V_bias, self.P_TES, label='TES PV')
        plt.title("Power Curve")
        ax.set_xlabel("Voltage (mV)")
        ax.set_ylabel("TES Power (fW)")
        
        if self.superconducting:
            readout = self.power_readout()
            plt.axhline(readout, color='g',label=round(readout,5))
        
        plt.legend()
        
    def plotRegions(self, raw = True):
        '''
        Currently only works for supercondicting
        '''
        fig, ax = plt.subplots()
        if raw:
            ohmic_data = np.append(self.rawI[:(self.ohmic_reg[0][1])], self.rawI[self.ohmic_reg[1][0]:])
            ohmic_V = np.append(self.rawV[:(self.ohmic_reg[0][1])],self.rawV[self.ohmic_reg[1][0]:])

            trans_data = np.append(self.rawI[self.trans_reg[0][0]: self.trans_reg[0][1]], self.rawI[self.trans_reg[1][0]:self.trans_reg[1][1]])
            trans_V = np.append(self.rawV[self.trans_reg[0][0]: self.trans_reg[0][1]], self.rawV[self.trans_reg[1][0]:self.trans_reg[1][1]])

            super_data = self.rawI[self.super_reg[0][0]:self.super_reg[0][1]]
            super_V = self.rawV[self.super_reg[0][0]:self.super_reg[0][1]]
            
            plt.scatter(ohmic_data, ohmic_V, label='Ohmic')
            plt.scatter(trans_data, trans_V, label='Transition')
            plt.scatter(super_data, super_V, label='Superconducting')
            ax.set_xlabel("I Bias (mA)")
            ax.set_ylabel("V Feedback (V)")
            plt.title("Raw IV Curve")
            plt.grid()
            plt.legend()
            plt.show()
        else:
            ohmic_data = np.append(self.I_TES[:(self.ohmic_reg[0][1])], self.I_TES[self.ohmic_reg[1][0]:])
            ohmic_V = np.append(self.V_bias[:(self.ohmic_reg[0][1])],self.V_bias[self.ohmic_reg[1][0]:])

            trans_data = np.append(self.I_TES[self.trans_reg[0][0]: self.trans_reg[0][1]], self.I_TES[self.trans_reg[1][0]:self.trans_reg[1][1]])
            trans_V = np.append(self.V_bias[self.trans_reg[0][0]: self.trans_reg[0][1]], self.V_bias[self.trans_reg[1][0]:self.trans_reg[1][1]])

            super_data = self.I_TES[self.super_reg[0][0]:self.super_reg[0][1]]
            super_V = self.V_bias[self.super_reg[0][0]:self.super_reg[0][1]]
            
            plt.scatter(ohmic_V, ohmic_data, label='Ohmic')
            plt.scatter(trans_V, trans_data, label='Transition')
            plt.scatter(super_V, super_data, label='Superconducting')
            
            ax.set_xlabel("Voltage (mV)")
            ax.set_ylabel("TES Current (mA)")
            plt.title("Calibrated IV Curve")
            plt.grid()
            plt.legend()
            plt.show()
        
        
        
    ###############Methods TODO###################
    def smooth_IV(): #Remove qunatum jumps?
        #TODO
        ''' Maybe a function to smooth noisy data before we find the 
        y offset. 
        Inputs: rawIV (or just use self?)
        Outputs: stores smoothRawIV in self
        '''
        pass
    
    def reject_outliers(self, data, m = 2.):
        d = np.abs(data - np.median(data))
        mdev = np.median(d)
        s = d/mdev if mdev else 0.
        return data[s<m]
    
    def intercect_R_stray(self):
        ''' Requires that there is an ohmic and a super conducting region.
        Removes x and y offsets by calcuating the intercect between the two lines, this is then used to 
        calculate R_stray
        '''
        self.calc_y_offset()
        self.calc_y_offset_ohmic()
        
        x = (self.D - self.F)/(self.E - self.C)
        y = self.C * (x) + self.D
        
        self.intercept = (x,y)
        self.x_offset = y
        self.y_offset = -0.001487731933594
 #x
        print('Y_offset in intercect method')
        print(x)
        
        C, D , Axvals1 = self.fit_ohmic(self.rawI, self.rawV)
        E, F , Axvals2 = self.fit_super(self.rawI, self.rawV)
        
        
        plt.scatter(self.rawI, self.rawV)
        plt.plot(Axvals1, self.f(Axvals1, C , D), color='r', label='Ohmic Line')
        plt.plot(Axvals2, self.f(Axvals2, E , F), color='g', label='Super Line')
        plt.show()
        
        self.newI = [entry - y for entry in self.rawI]
        self.newV = [entry - x for entry in self.rawV]
        self.calc_I_TES()
        self.calc_R_stray()
        
    def y_search(self, step, pivot, precision):

        ys = []
        counts = []
        
        if step <= precision:
            self.y_offset = pivot
            return pivot
        
        
        for y_off in np.arange(pivot-(10*step),pivot+(step*10),step/10):
            I_TES = [(entry - y_off) * (10**6) * self.gain for entry in self.rawV]
            V_bias = [((self.rawI[i]) * self.R_shunt) - (I_TES[i]*(self.R_shunt + self.R_stray)) for i in range(len(self.rawI))]
            P_TES = [I_TES[index] * V_bias[index] for index in range(len(self.I_TES))]
            P_grad = np.gradient(P_TES)
            P_grad_smooth = self.reject_outliers(P_grad)
            R_TES = [V_bias[i] / I_TES[i] for i in range(len(V_bias))]
            count =0
            
            max_R = max(R_TES)
            
            for entry in P_grad_smooth:
                count += 5 * abs(entry)**2
            for i in range(int(len(R_TES)/2)-1):
                if (R_TES[i] > 0.95*max_R or R_TES[-i] > 0.95*max_R):
                    count += 2*(R_TES[i] - R_TES[-i])**2
            
            if np.average(R_TES) > 0:
                counts.append(count)
                ys.append(y_off)
        
        y_offset = ys[np.argmin(counts)]
        

        return self.y_search(step/10, y_offset, precision)
        
    def reverse_y_offset(self):
        '''Requires R_stray to have been already calculated or input.
        Attempts to calc offset through optimising the power plot. 
        '''
        self.y_search(5,0,0.000001)
        
    def calc_all(self):
        self.calc_I_TES()
        self.calc_V_bias()

        self.plotIV(False)
        self.calc_P_TES()
        self.calc_R_TES()

In [8]:
class IV_series(object):
    
    R_nn = 0
    R_stray = 0 
    
    def __init__(self, dirname, params):
        read_data(dirname)
        self.data = [TES(series, params) for series in self.series]
        
        
    def calc_R_stray(self, value = False):
        if value:
            R_stray = value
        else:
            pass # TODO impliment 
        
        
    def calc_R_nn(self):
        average = 0
        for IV_curve in self.data:
            IV_curve.plotIV(False)
            average += IV_curve.rnn
        
        R_nn = average/len(self.data)
    
    
    def read_data(dirname):
        series = []
        for filename in os.listdir(dirname):
            series.append(read_qdp(filename))
        self.series = series    
        

In [9]:
params = [3002.9, 18.93333, 0.001115]
RT = TES(data4, params, 0.0005)

In [11]:
print(RT.T_BB)

[]


In [12]:
print(RT.rawI)

[-111.991719 -111.007737 -110.005533 -109.003329 -108.001126 -106.998922
 -105.996718 -104.994514 -103.992311 -103.008329 -102.006125 -101.003921
 -100.001717  -98.999513  -97.99731   -96.995106  -95.992902  -95.00892
  -94.006716  -93.004512  -92.002309  -91.000105  -89.997901  -88.995697
  -87.993494  -86.99129   -86.007308  -85.005104  -84.0029    -83.000696
  -81.998493  -80.996289  -79.994085  -75.001288  -70.008491  -64.997472
  -60.004675  -54.993656  -50.000859  -45.008061  -39.997043  -35.004245
  -32.999838  -29.993226  -25.000429  -23.998226  -22.996022  -21.993818
  -20.991614  -20.007632  -19.005428  -18.003225  -17.001021  -15.998817
  -14.996613  -10.003816   -4.992797    0.          4.992797   10.003816
   14.996613   15.998817   17.001021   18.003225   19.005428   20.007632
   20.991614   21.993818   22.996022   23.998226   25.000429   29.993226
   32.999838   35.004245   39.997043   45.008061   50.000859   54.993656
   60.004675   64.997472   70.008491   75.001288   7

In [22]:
params17D = [30002.9, 18.93333, 0.001278]
dataDict = pd.read_excel(r'C:\Users\callu\Dropbox\University\Year 3\Block 2b\Project\Data\17D_CBB_low.xls', sheet_name=None)
mes_set_17D = [TES(sheet, params17D, name) for name, sheet in dataDict.items()]

In [23]:
print(mes_set_17D[0].T_bath)


[[69.97979]
 [69.99166]
 [69.99166]
 [70.00352]
 [69.99166]
 [70.02726]
 [70.00352]
 [70.00352]
 [70.00352]
 [69.97979]
 [69.99166]
 [69.99166]
 [69.99166]
 [69.99166]
 [69.97979]
 [70.02726]
 [70.00352]
 [70.01539]
 [70.00352]
 [70.00352]
 [70.00352]
 [70.00352]
 [70.00352]
 [69.97979]
 [70.00352]
 [70.00352]
 [70.00352]
 [69.99166]
 [70.00352]
 [69.99166]
 [69.99166]
 [69.99166]
 [69.99166]
 [70.00352]
 [69.99166]
 [69.99166]
 [69.99166]
 [70.01539]
 [69.99166]
 [70.01539]
 [70.00352]
 [69.99166]
 [69.97979]
 [69.99166]
 [70.00352]
 [70.00352]
 [70.01539]
 [69.97979]
 [70.01539]
 [69.99166]
 [69.99166]
 [70.00352]
 [69.97979]
 [69.99166]
 [70.00352]
 [70.00352]
 [69.99166]
 [69.97979]
 [69.99166]
 [70.00352]
 [69.99166]
 [69.99166]
 [70.00352]
 [69.99166]
 [70.00352]
 [70.00352]
 [70.00352]
 [69.99166]
 [69.99166]
 [69.99166]
 [70.00352]
 [69.99166]
 [70.00352]
 [69.99166]
 [69.99166]
 [70.02726]
 [70.00352]
 [69.99166]
 [70.00352]
 [69.99166]
 [70.00352]
 [70.00352]
 [69.99166]
 [70

In [24]:
print(mes_set_17D[0].T_BB)

[3.169203, 3.169328, 3.169328, 3.169203, 3.169164, 3.168956, 3.168622, 3.168414, 3.168163, 3.167873, 3.1675, 3.167041, 3.16675, 3.166833, 3.16675, 3.166668, 3.166625, 3.166958, 3.166876, 3.167206, 3.167457, 3.167873, 3.167998, 3.168163, 3.168289, 3.168038, 3.168038, 3.168246, 3.168163, 3.168081, 3.168163, 3.167916, 3.167916, 3.167873, 3.167622, 3.167374, 3.167123, 3.167084, 3.1667899999999998, 3.1667899999999998, 3.1667899999999998, 3.166915, 3.167041, 3.167249, 3.167457, 3.1675, 3.167582, 3.167916, 3.167873, 3.168038, 3.168124, 3.168038, 3.168332, 3.168414, 3.168414, 3.168579, 3.168661, 3.168787, 3.168748, 3.168869, 3.168956, 3.168912, 3.168956, 3.168956, 3.168956, 3.168787, 3.168787, 3.16883, 3.168787, 3.168622, 3.16854, 3.168453, 3.168206, 3.168289, 3.168081, 3.167916, 3.167873, 3.167873, 3.16779, 3.167665, 3.167331, 3.1672919999999998, 3.167123, 3.167123, 3.166876, 3.166707, 3.166707, 3.166876, 3.167041, 3.167331, 3.167539, 3.167747, 3.168081, 3.168038, 3.167955, 3.168163, 3.168163